## 데이콘 비트코인 가격 예측 - Neural Prophet pilot

### library install & import, data load

In [38]:
# import library
from datetime import timedelta, datetime
from copy import deepcopy
import datetime
import argparse
import random
import warnings


import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid

from tqdm import tqdm
import matplotlib.pyplot as plt

# model
from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed


%matplotlib inline
warnings.filterwarnings(action = "ignore")

# read file
train_x_df = pd.read_csv("./data/train_x_df.csv")
train_y_df = pd.read_csv("./data/train_y_df.csv")
print("Reading Complete!")


Reading Complete!


### data preprocess functions

In [39]:
def df2d_to_array3d(df_2d):

    ''' 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수 '''

    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    
    return array_3d
    

def coin_index_export(input_array, coin_num):

    ''' 코인별 인덱스를 뽑아 list형식으로 만들어주는 함수 '''
    
    index = []
    sample_id_len = input_array.shape[0]
    coin_num_col = 0 

    for sample_id in range(sample_id_len):
        if input_array[sample_id, 0, coin_num_col] == coin_num:
            #print(sample_id)
            index.append(sample_id)
    
    return index


def prophet_preprocessor(x_series):
    
    # start time initialization
    start_time = '2021-01-01 00:00:00'
    start_dt = datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')

    # datafram 만들기
    x_df = pd.DataFrame()
    # 분당 시간 데이터 시리즈 입력
    x_df['ds'] = [start_dt + datetime.timedelta(minutes = time_min) for time_min in np.arange(1, x_series.shape[0]+1).tolist()]
    # 가격 데이터 시리즈 입력
    x_df['y'] = x_series.tolist()

    return x_df

### parameter setting

In [43]:
params_grid = {'n_changepoints':[5, 10, 15, 20], #default 5
                'changepoints_range' : [0.9, 0.95, 1.0], #defalut 0.8
                'num_hidden_layers' : [0, 1, 2],
                'd_hidden' : [10, 30, 50],

}

grid = ParameterGrid(params_grid)
cnt = 0
for parameters in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)


Total Possible Models 108


### grid_search

In [55]:
# params_grid = {
# 'seasonality_fourier_order' : [[1, 5], [5, 10], [10,15], [15, 20]],
# 'seasonality_periods' : [[1/12, 1/8], [1/12, 1/6], [1/12, 1/4], [1/24, 1/16], [1/24, 1/12], [1/24, 1/8]]
# }

# grid = ParameterGrid(params_grid)
# cnt = 0
# for parameters in grid:
#     cnt = cnt+1

# print('Total Possible Models',cnt)


In [46]:
def train(x_series, y_series, **paras):
    
    x_df = prophet_preprocessor(x_series)
    
    model = NeuralProphet(**paras,
#                           n_changepoints = paras['n_changepoints'],
#                           changepoints_range = paras['changepoints_range'],
#                           num_hidden_layers = paras['num_hidden_layers'],
#                             trend_reg = 0.05, seasonality_reg = 0.05,
        
                          learning_rate = 0.1, epochs=40, batch_size = 32, loss_func="Huber",
                          seasonality_mode = 'multiplicative',
                          yearly_seasonality = False, weekly_seasonality = False, daily_seasonality = False,
                          normalize='minmax'# Type of normalization ('minmax', 'standardize', 'soft', 'off')
                         )
    
    model.add_seasonality(name='first_seasonality', period=1/24, fourier_order=5) 
    model.add_seasonality(name='second_seasonality', period=1/12, fourier_order=10)

    metrics = model.fit(x_df, freq="min")

    future = model.make_future_dataframe(x_df, periods=120)
    forecast = model.predict(future)
    error = mean_squared_error(y_series, forecast.yhat1.values[-120:])

    return error


In [47]:

def experiment(train_x_array, train_y_array, args):
    
    sample_indices = coin_index_export(input_array = train_x_array, coin_num= args.coin_num)
    args.sample_id_indices = random.sample(sample_indices, 5)
    print(args.sample_id_indices)

    
    result = pd.DataFrame(columns = ['MSE*10E5', 'PARAMETERS'])

    for paras in grid:
        print(f'\n current parameters is {paras}\n')
        total_mse = 0.0

        for sample_id in args.sample_id_indices:

            x_series = train_x_array[sample_id, :, args.data_col_idx] 
            y_series = train_y_array[sample_id, :, args.data_col_idx]

            temp_mse = train(x_series, y_series, **paras)
            total_mse += temp_mse * 10E5
            
        print('\nTotal MSE Error------------------------------------', total_mse, end='\n')
        result = result.append({'MSE*10E5':total_mse, 'PARAMETERS':paras},ignore_index=True)


    return result

In [48]:
train_x_array = df2d_to_array3d(train_x_df)
train_y_array = df2d_to_array3d(train_y_df)

seed = 7

np.random.seed(seed)
set_random_seed(seed)
random.seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.data_col_idx = 1 # 1 open, 2 high
args.coin_num = 9


In [49]:
result = experiment(train_x_array, train_y_array, deepcopy(args))

Epoch[1/40]:   2%|▋                         | 1/40 [00:00<00:04,  7.96it/s, SmoothL1Loss=0.185, MAE=0.00607, RegLoss=0]

[4176, 1934, 5025, 601, 947]

 current parameters is {'changepoints_range': 0.9, 'd_hidden': 10, 'n_changepoints': 5, 'num_hidden_layers': 0}



Epoch[2/40]:   0%|                                    | 0/40 [00:00<?, ?it/s, SmoothL1Loss=1.26, MAE=0.0204, RegLoss=0]


Total MSE Error------------------------------------ 149.7319014226228

 current parameters is {'changepoints_range': 0.9, 'd_hidden': 10, 'n_changepoints': 5, 'num_hidden_layers': 1}



Epoch[1/40]:   2%|▋                          | 1/40 [00:00<00:04,  8.65it/s, SmoothL1Loss=0.14, MAE=0.00546, RegLoss=0]

...


Total MSE Error------------------------------------ 240.59713998205754

 current parameters is {'changepoints_range': 1.0, 'd_hidden': 50, 'n_changepoints': 20, 'num_hidden_layers': 1}



Epoch[2/40]:   5%|█▎                        | 2/40 [00:00<00:03, 12.08it/s, SmoothL1Loss=0.154, MAE=0.00605, RegLoss=0]


Total MSE Error------------------------------------ 203.27440719087897

 current parameters is {'changepoints_range': 1.0, 'd_hidden': 50, 'n_changepoints': 20, 'num_hidden_layers': 2}



Epoch[40/40]: 100%|██████████████████████| 40/40 [00:03<00:00, 11.13it/s, SmoothL1Loss=0.00104, MAE=0.00139, RegLoss=0]


Total MSE Error------------------------------------ 273.4156465341368


In [50]:
result = result.sort_values(by =['MSE*10E5'])
result = result.reset_index(drop=True)


In [51]:
result.head()

,MSE*10E5,PARAMETERS
0,90.089789,"{'changepoints_range': 0.95, 'd_hidden': 30, '..."
1,90.955719,"{'changepoints_range': 0.95, 'd_hidden': 30, '..."
2,91.008956,"{'changepoints_range': 0.95, 'd_hidden': 10, '..."
3,91.268299,"{'changepoints_range': 0.95, 'd_hidden': 10, '..."
4,94.842062,"{'changepoints_range': 0.9, 'd_hidden': 30, 'n..."


In [52]:
result.to_csv('hyperparameteer_grid_search.csv')

In [53]:
result['PARAMETERS'][0]

{'changepoints_range': 0.95,
 'd_hidden': 30,
 'n_changepoints': 20,
 'num_hidden_layers': 1}

In [35]:
1/24, 1/12

(0.041666666666666664, 0.08333333333333333)

In [77]:
# plt.plot(np.arange(1380, 1380+120), y_series, label = 'True series')
# plt.plot(np.arange(1380, 1380+120), forecast.yhat1.values[-120:], label = 'Neural Prophet1')
# plt.legend()
# plt.show()

# error = mean_squared_error(y_series, forecast.yhat1.values[-120:])
# print("Prophet MSE*10E5 is {}".format(error * 10E5))
